# MLP Regressor

**Multi-Layer Perceptron (MLP) Regressor** is a type of artificial neural network used in supervised learning tasks where the goal is to predict a continuous dependent variable. MLPs are capable of learning complex patterns using multiple layers and neurons.

## Concept

MLP consists of at least three layers of nodes: an input layer, a hidden layer, and an output layer. Except for the input nodes, each node is a neuron that uses a nonlinear activation function. MLP utilizes a technique called backpropagation for training the network.

## Architecture

- **Input Layer:** Receives the feature set as inputs which are then passed to the first hidden layer.
- **Hidden Layers:** Each hidden layer transforms the inputs from the previous layer based on weights, biases, and activation functions. MLP can have one or more hidden layers.
- **Output Layer:** Produces the final output of the network. In the case of regression, it typically has a single neuron for single-target regression or multiple neurons for multi-target regression.

## Key Parameters

- $n_{\text{hidden}}$: Number of neurons in the hidden layers.
- $\text{activation}$: The activation function for the neurons, typically ReLU, sigmoid, or tanh.
- $\text{solver}$: The algorithm for weight optimization, such as SGD, Adam, or LBFGS.
- $\text{learning\_rate}$: Determines the step size at each iteration while moving toward a minimum of a loss function.
- $\text{max\_iter}$: Maximum number of iterations. The solver iterates until convergence or this number of iterations.
- $\alpha$: L2 penalty (regularization term) parameter. A higher value of $\alpha$ increases the regularization strength, which helps reduce overfitting but can make the network less sensitive to the training data.


## Training Process

1. **Forward Propagation:** Compute the predicted output $\hat{y}$, using the current weights and biases in the network layers.
2. **Calculate Loss:** The difference between the actual output $y$ and predicted output $\hat{y}$ using a loss function, typically Mean Squared Error (MSE) for regression.
3. **Backpropagation:** Update the weights and biases in the network in a way that minimizes the loss.

## Advantages

- Capable of modeling highly non-linear functions.
- Flexible to model various types of responses through the choice of activation and loss functions.
- Suitable for large datasets and complex feature interactions.

## Applications

MLP is widely used in:
- Predicting energy consumption,
- Stock market predictions,
- Real estate price forecasting,
- Any complex regression tasks in quantitative finance and economics.


# Implementation

### Import Libraries

In [4]:
# import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from IPython.display import display, clear_output, HTML

import warnings
warnings.filterwarnings("ignore")

### Import and show Data

In [5]:
data = pd.read_csv('./Data/BeadArea.csv')
display(data.head())
print ("The data is composed of ", data.shape[0], " rows and ", data.shape[1], " columns.")

,Time,Position Feedback,Velocity Feedback from Axis 1,Velocity Feedback from Axis 2,Temperature,Current Feedback,Interpolated Bead Area
0,0.056,15.567,11.178,6.4952,1.1173,1.2309,1.1184
1,0.057,15.579,11.257,6.4952,1.1175,1.2735,1.1191
2,0.058,15.590,11.375,3.2476,1.1175,1.1567,1.1198
3,0.059,15.601,11.324,6.4952,1.1173,1.2975,1.1205
4,0.060,15.613,11.285,6.4952,1.1173,1.2168,1.1212


The data is composed of  295568  rows and  7  columns.


### Data Preprocessing

In [6]:
data['Lag1'] = data['Interpolated Bead Area'].shift(1)
data['Lag2'] = data['Interpolated Bead Area'].shift(2)
data['Lag3'] = data['Interpolated Bead Area'].shift(3)
data.dropna(inplace=True)

### Predict Bead Area

In [7]:
# Define widgets with adjusted layout
index_range_slider = widgets.IntRangeSlider(
    value=[0, min(5000, len(data))],
    min=0,
    max=len(data),
    step=1,
    description='Index Range:',
    layout=widgets.Layout(width='600px'),  # Increase width for better readability
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

feature_select = widgets.SelectMultiple(
    options=['Lag1', 'Lag2', 'Lag3', 'Time', 'Position Feedback', 'Velocity Feedback from Axis 1', 'Velocity Feedback from Axis 2', 'Temperature', 'Current Feedback'],
    value=['Lag1', 'Lag2', 'Lag3', 'Time', 'Position Feedback', 'Velocity Feedback from Axis 1', 'Velocity Feedback from Axis 2', 'Temperature', 'Current Feedback'],
    description='Features:',
    layout=widgets.Layout(width='600px', height='180px'),  # Increase width and height
    style={'description_width': '150px'},  # Increase description width
    disabled=False
)

train_size_slider = widgets.IntSlider(
    value=80,
    min=50,
    max=95,
    step=1,
    description='Train %:',
    layout=widgets.Layout(width='600px'),  # Increase width
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

# MLP Regressor parameter sliders
hidden_layer_sizes_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=500,
    step=10,
    description='Hidden Layer Sizes:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False
)

activation_dropdown = widgets.Dropdown(
    options=['identity', 'logistic', 'tanh', 'relu'],
    value='relu',
    description='Activation Function:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False
)

alpha_slider = widgets.FloatSlider(
    value=0.0001,
    min=0.00001,
    max=0.1,
    step=0.00001,
    description='L2 regularization:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False
)

apply_button = widgets.Button(description="Apply Changes", layout=widgets.Layout(width='800px'))

# Define the function to apply changes and update the plots
def apply_changes(b):
    with output:
        clear_output(wait=True)
        
        # Extract the parameters from widgets
        index_range = index_range_slider.value
        selected_features = list(feature_select.value)
        train_size_pct = train_size_slider.value / 100
        hidden_layer_sizes = hidden_layer_sizes_slider.value
        activation = activation_dropdown.value
        alpha = alpha_slider.value
        
        # Slice the data
        df = data[index_range[0]:index_range[1]]
        
        # Prepare the data (assuming 'Interpolated Bead Area' is already in `df`)
        X = df[selected_features]
        y = df['Interpolated Bead Area']
        
        # Train-test split
        train_size = int(len(df) * train_size_pct)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        # Train the model
        model = MLPRegressor(
            hidden_layer_sizes=(hidden_layer_sizes,),
            activation=activation,
            alpha=alpha,
            random_state=42,
            max_iter=100000
        )
        model.fit(X_train, y_train)
        
        # Predict on test data
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        display(HTML(f'<b>Mean Squared Error: {mse:.5f}</b>'))  # Display MSE in bold
        
        # Plot predicted vs actual
        plt.figure(figsize=(10, 6))
        plt.plot(y_train.index, y_train, label='Training', color='green')
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.xlabel('Time')
        plt.ylabel('Interpolated Bead Area')
        plt.title('Actual vs Predicted Interpolated Bead Area')
        plt.legend()
        plt.show()
        
        # Calculate loss for each point
        pointwise_mse_loss = (y_test - y_pred) ** 2
        
        # Plot the pointwise loss
        plt.figure(figsize=(10, 6))
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.plot(y_test.index, pointwise_mse_loss, label='Pointwise MSE Loss', color='orange')
        plt.xlabel('Time')
        plt.ylabel('MSE Loss')
        plt.title('Pointwise MSE Loss of Predicted vs Actual Interpolated Bead Area')
        plt.legend()
        plt.show()

# Link the apply button to the function
apply_button.on_click(apply_changes)

# Display the widgets and the output area
output = widgets.Output()

display(index_range_slider, feature_select, train_size_slider, hidden_layer_sizes_slider, activation_dropdown, alpha_slider, apply_button, output)


IntRangeSlider(value=(0, 5000), continuous_update=False, description='Index Range:', layout=Layout(width='600p…

SelectMultiple(description='Features:', index=(0, 1, 2, 3, 4, 5, 6, 7, 8), layout=Layout(height='180px', width…

IntSlider(value=80, continuous_update=False, description='Train %:', layout=Layout(width='600px'), max=95, min…

IntSlider(value=100, continuous_update=False, description='Hidden Layer Sizes:', layout=Layout(width='600px'),…

Dropdown(description='Activation Function:', index=3, layout=Layout(width='600px'), options=('identity', 'logi…

FloatSlider(value=0.0001, continuous_update=False, description='L2 regularization:', layout=Layout(width='600p…

Button(description='Apply Changes', layout=Layout(width='800px'), style=ButtonStyle())

Output()